In [7]:
import pandas as pd
import numpy as np
from glob import glob
from tqdm import tqdm
import os
from sklearn.model_selection import train_test_split
import json
from math import sin, cos
import imutils
from skimage.transform import rotate

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torchvision import datasets, models, transforms
from torchvision.transforms.autoaugment import AutoAugmentPolicy

import pickle 

import cv2
from PIL import Image

import matplotlib.pyplot as plt
from IPython.display import clear_output

In [19]:
with open('submissions/wideresnet_trig_ext_relu_uncloud.pickle', 'rb') as f:
    resnet_arr = pickle.load(f)

In [21]:
with open('submissions/resnext_trig_ext_relu_4.pickle', 'rb') as f:
    densnet_arr = pickle.load(f)

In [22]:
test_files = sorted(glob('data\\test\\img_uncloud\\*'))

In [23]:
indexes = [x.split('\\')[-1].split('.')[0] for x in test_files]
preds = []

In [25]:
sub_dir = "submissions\\subm_stack_wideres_resnext_uncloud_ext_trig\\"
if not os.path.exists(sub_dir):
    os.makedirs(sub_dir)

json_true = []

for indx, pr1, pr2 in zip(indexes, resnet_arr, densnet_arr):
    #print(indx, pr1, pr2)
    pred = (pr1+pr2)/2
    pred[:-1] = np.clip(pred[:-1], 0, 10496)
    pred[-1] = np.clip(pred[-1], 0, 360)
#     cx = (pred[0]+pred[2])/2
#     cy = (pred[1]+pred[3])/2
    pred = [int(x) for x in pred]
    pred[-1] = pred[-1]%360
    left_top = [pred[0], pred[1]]
    right_top = [pred[6], pred[7]]
    left_bottom = [pred[4], pred[5]]
    right_bottom = [pred[2], pred[3]]
        
    res = {
        'left_top': left_top,
        'right_top': right_top,
        'left_bottom': left_bottom,
        'right_bottom': right_bottom,
        'angle': pred[4]
    }

    with open(sub_dir+indx+'.json', 'w') as f:
        json.dump(res, f)

In [36]:
sub_dir = "submissions\\subm_stack_incept_res_dense\\"
if not os.path.exists(sub_dir):
    os.makedirs(sub_dir)

json_true = []

for indx, pr1, pr2,pr3 in zip(indexes, resnet_arr, densnet_arr, inception_arr):
    #print(indx, pr1, pr2)
    pred = (pr1+pr2+pr3)/3
    pred[:4] = np.clip(pred[:4], 0, 10496)
    pred[-1] = np.clip(pred[-1], 0, 360)
    pred = [int(x) for x in pred]

    left_top = [pred[0], pred[1]]
    right_top = [pred[2], pred[1]]
    left_bottom = [pred[0], pred[3]]
    right_bottom = [pred[2], pred[3]]
        
    res = {
        'left_top': left_top,
        'right_top': right_top,
        'left_bottom': left_bottom,
        'right_bottom': right_bottom,
        'angle': pred[4]
    }

    with open(sub_dir+indx+'.json', 'w') as f:
        json.dump(res, f)